In [14]:
# Jupyter-Notebook: 2D Navier–Stokes (FEM, NGSolve) – Kanalströmung um Zylinder (Strudelstraße)
# Benötigt: ngsolve, netgen. In Jupyter: zusätzlich ngsolve.webgui für inline Visualisierung.

from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw
import math


In [15]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw

L, H = 2.2, 0.41
cx, cy, r = 0.2, 0.2, 0.05

geo = SplineGeometry()

# Punkte
p1 = geo.AddPoint(0,   0)
p2 = geo.AddPoint(L,   0)
p3 = geo.AddPoint(L,   H)
p4 = geo.AddPoint(0,   H)

# Kanalränder  (WICHTIG: Punkte als Liste!)
geo.AddSegment([p1, p2], bc="walls")    # unten
geo.AddSegment([p2, p3], bc="outlet")   # rechts
geo.AddSegment([p3, p4], bc="walls")    # oben
geo.AddSegment([p4, p1], bc="inlet")    # links

# Zylinder (Loch)
geo.AddCircle((cx, cy), r=r, bc="cyl", leftdomain=0, rightdomain=1)

mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)

Draw(mesh)



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [16]:
# -----------------------------
# 2) Räume (Taylor–Hood): V = (H1)^2 Ordnung 2, Q = L2 Ordnung 1
#    Dirichlet (no-slip / inflow): inlet|walls|cyl
# -----------------------------
order_u = 2
order_p = 1

V = VectorH1(mesh, order=order_u, dirichlet="inlet|walls|cyl")
Q = L2(mesh, order=order_p)  # Druck in L2 (Mean-0 wird später fixiert)
X = V * Q

(u, p) = X.TrialFunction()
(v, q) = X.TestFunction()

gfu = GridFunction(X)   # (u,p) gesamt
un, pn = gfu.components  # aktuelle Lösung


In [ ]:
# -----------------------------
# 3) Physik/Parameter
#    Reynolds-Zahl ~ 100: nu = U*D/Re.
#    Wir normieren mit U_max=1, D=2r=0.1 => nu = U*D/Re = 0.1/100 = 0.001
# -----------------------------
Re = 100.0
D  = 2*r
Umax = 1.0
nu = Umax*D/Re  # kinematische Viskosität

dt = 0.002
t_end = 4.0

# Parabolisches Inflow-Profil (Poiseuille) an x=0: u_x(y)=4*U_mean*y*(H-y)/H^2
# Hier nehmen wir U_mean so, dass max etwa Umax ist: u_max = 1.5*U_mean => U_mean=Umax/1.5
Umean = Umax/1.5
y = y  # NGSolve symbol
uin = CoefficientFunction( (4*Umean*y*(H-y)/(H*H), 0) )


In [27]:
# -----------------------------
# 4) Randwerte setzen (Dirichlet auf inlet|walls|cyl)
#    - inlet: u = uin
#    - walls, cyl: u = (0,0)
# -----------------------------
un.Set(uin, definedon=mesh.Boundaries("inlet"))
un.Set((0,0), definedon=mesh.Boundaries("walls|cyl"))





In [28]:
# -----------------------------
# 5) Zeitdiskretisierung (linearisiert / Oseen):
#    (u^{n+1}-u^n)/dt + (u^n · ∇) u^{n+1} - nu Δ u^{n+1} + ∇p^{n+1} = 0
#    div u^{n+1} = 0
#
#    Outlet: "do-nothing" natürlich durch schwache Form (kein Dirichlet am outlet).
# -----------------------------
uold = GridFunction(V)
uold.vec.data = un.vec  # Startwert

n = specialcf.normal(mesh.dim)

def gradv(w):  # symmetrischer Gradient (optional); hier: voller grad für Standard-NS
    return grad(w)

# Konvektion: (uold · ∇) u
a = BilinearForm(X, symmetric=False)

a += (1/dt) * InnerProduct(u, v) * dx
a += nu * InnerProduct(grad(u), grad(v)) * dx
a += InnerProduct(grad(u) * uold, v) * dx     # ← KORREKT
a += (-div(v) * p - div(u) * q) * dx


# Optional: kleiner Druck-Stabilisierer, falls nötig (meist nicht für Taylor–Hood)
# a += 1e-12*p*q*dx

f = LinearForm(X)
f += (1/dt)*InnerProduct(uold, v)*dx

# Matrix/Preconditioner
a.Assemble()
f.Assemble()

# Direkter Solver (robust, für kleinere Meshes ok)
inv = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")


In [29]:
# -----------------------------
# 6) Visualisierung (inline)
# -----------------------------
scene_u = Draw(un, mesh, "velocity", autoscale=False)
scene_p = Draw(pn, mesh, "pressure")
scene_u


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [30]:
# -----------------------------
# 7) Zeitschleife
#    Tipp: Wenn du Strudelstraße “sicher” sehen willst:
#    - Mesh verfeinern (maxh ~ 0.01 in Zylindernähe)
#    - dt kleiner (z.B. 0.001)
#    - t_end größer (z.B. 8..12)
# -----------------------------
t = 0.0
step = 0

while t < t_end - 1e-12:
    # RHS aus uold neu aufbauen
    f = LinearForm(X)
    f += (1/dt)*InnerProduct(uold, v)*dx
    f.Assemble()


    # Konvektion hängt von uold ab -> a neu assemblieren
    a = BilinearForm(X, symmetric=False)
    a += (1/dt)*InnerProduct(u, v)*dx
    a += nu*InnerProduct(grad(u), grad(v))*dx
    a += InnerProduct(grad(u) * uold, v)*dx
    a += (-div(v)*p - div(u)*q)*dx
    a.Assemble()

    inv = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

    gfu.vec.data = inv * f.vec

    # Dirichlet-Werte wieder erzwingen (inlet/walls/cyl)
    un.Set(uin, definedon=mesh.Boundaries("inlet"))
    un.Set((0,0), definedon=mesh.Boundaries("walls|cyl"))

    # Update uold
    uold.vec.data = un.vec

    t += dt
    step += 1

    if step % 20 == 0:
        scene_u.Redraw()
        scene_p.Redraw()

t, step


KeyboardInterrupt: 

In [ ]:
G = grad(un)          # 2×2 Matrixfeld
omega = G[1,0] - G[0,1]
Draw(omega, mesh, "vorticity")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene